In [7]:
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, collect_list, udf
from collections import defaultdict
import numpy as np

from pyspark.sql.session import SparkSession
from pyspark import SparkContext
import dill
import os.path

In [9]:
# spark = SparkSession.builder \
#     .appName("concrec-recall") \
#     .master("local[*]") \
#     .config("spark.driver.bindAddress", "127.0.0.1") \
#     .config("spark.ui.port", "4050") \
#     .getOrCreate()
spark = SparkSession.builder.appName("concrec-recall").getOrCreate()

24/10/29 11:58:01 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1074468 ms exceeds timeout 120000 ms
24/10/29 11:58:01 WARN SparkContext: Killing executors is not supported by current scheduler.
24/10/29 12:17:53 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [3]:
BASE_PATH = '../dataset/'
## DataFrame
rating_df = spark.read.csv(BASE_PATH + 'rating.csv', header=True, inferSchema=True)
rating_df.printSchema()
# rating_df = rating_df.where('rating > 7')
rating_df = rating_df[rating_df['rating'] > 7]
# print(rating_df[1])   ## Column<'anime_id'>
# print(rating_df.collect()[1])    ## change to list: Row(user_id=1, anime_id=11617, rating=10)
print(len(rating_df.collect()))    ## 3855830

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



3855830


In [4]:
## user_id <=> [anime_id] group by user_id
## anime_id: 转换为string 为了后面的方便
watch_seq_df = rating_df \
    .groupBy('user_id') \
    .agg(
        collect_list(col('anime_id').cast('string')).alias('anime_ids')     
    )

In [5]:
## bring it to the driver node as a Python list.
watch_seq = watch_seq_df.collect() ## two columns
print(watch_seq[1])

Row(user_id=27, anime_ids=['356', '1195', '1535', '1575', '1604', '1840', '2167', '2904', '2993', '3455', '3712', '4181', '4214', '4224', '4654', '6201', '6213', '6324', '6347', '6547', '6702', '7054', '7592', '7593', '8074', '8516', '8525', '8536', '8769', '8841', '8937', '9041', '9513', '9919', '10049', '10080', '10213', '10620', '10719', '10790', '10793', '11319', '11617', '11633', '11703', '11743', '11757', '11933', '12049', '13659', '13759', '14075', '14227', '14345', '14467', '14741', '14749', '14813', '14967', '15225', '15315', '15451', '15583', '15809', '16049', '16157', '16444', '16498', '16512', '16668', '16706', '17651', '17741', '18857'])


In [6]:
watch_seq = [r['anime_ids'] for r in watch_seq]
watch_seq[1]

['356',
 '1195',
 '1535',
 '1575',
 '1604',
 '1840',
 '2167',
 '2904',
 '2993',
 '3455',
 '3712',
 '4181',
 '4214',
 '4224',
 '4654',
 '6201',
 '6213',
 '6324',
 '6347',
 '6547',
 '6702',
 '7054',
 '7592',
 '7593',
 '8074',
 '8516',
 '8525',
 '8536',
 '8769',
 '8841',
 '8937',
 '9041',
 '9513',
 '9919',
 '10049',
 '10080',
 '10213',
 '10620',
 '10719',
 '10790',
 '10793',
 '11319',
 '11617',
 '11633',
 '11703',
 '11743',
 '11757',
 '11933',
 '12049',
 '13659',
 '13759',
 '14075',
 '14227',
 '14345',
 '14467',
 '14741',
 '14749',
 '14813',
 '14967',
 '15225',
 '15315',
 '15451',
 '15583',
 '15809',
 '16049',
 '16157',
 '16444',
 '16498',
 '16512',
 '16668',
 '16706',
 '17651',
 '17741',
 '18857']

In [7]:
## 邻接矩阵
matrix = defaultdict(lambda: defaultdict(int)) ## matrix['a']['b'] += 1

In [8]:
def add_seq_to_matrix(seq, matrix):     ## matrix[a][b]++ and matrix[b][a]++
    for i in range(len(seq)):
        for j in range(i + 1, len(seq)):
            a = seq[i]
            b = seq[j]
            if a == b:
                continue
            matrix[a][b] += 1
            matrix[b][a] += 1

In [9]:
add_seq_to_matrix(watch_seq[1], matrix)
print(matrix)

defaultdict(<function <lambda> at 0x15062ae80>, {'356': defaultdict(<class 'int'>, {'1195': 1, '1535': 1, '1575': 1, '1604': 1, '1840': 1, '2167': 1, '2904': 1, '2993': 1, '3455': 1, '3712': 1, '4181': 1, '4214': 1, '4224': 1, '4654': 1, '6201': 1, '6213': 1, '6324': 1, '6347': 1, '6547': 1, '6702': 1, '7054': 1, '7592': 1, '7593': 1, '8074': 1, '8516': 1, '8525': 1, '8536': 1, '8769': 1, '8841': 1, '8937': 1, '9041': 1, '9513': 1, '9919': 1, '10049': 1, '10080': 1, '10213': 1, '10620': 1, '10719': 1, '10790': 1, '10793': 1, '11319': 1, '11617': 1, '11633': 1, '11703': 1, '11743': 1, '11757': 1, '11933': 1, '12049': 1, '13659': 1, '13759': 1, '14075': 1, '14227': 1, '14345': 1, '14467': 1, '14741': 1, '14749': 1, '14813': 1, '14967': 1, '15225': 1, '15315': 1, '15451': 1, '15583': 1, '15809': 1, '16049': 1, '16157': 1, '16444': 1, '16498': 1, '16512': 1, '16668': 1, '16706': 1, '17651': 1, '17741': 1, '18857': 1}), '1195': defaultdict(<class 'int'>, {'356': 1, '1535': 1, '1575': 1, '16

In [10]:
for i in range(len(watch_seq)):
    if i % 1000 == 0:
        print(f'matrix add seq {i}')
    seq = watch_seq[i]
    add_seq_to_matrix(seq, matrix)

matrix add seq 0
matrix add seq 1000
matrix add seq 2000
matrix add seq 3000
matrix add seq 4000
matrix add seq 5000
matrix add seq 6000
matrix add seq 7000
matrix add seq 8000
matrix add seq 9000
matrix add seq 10000
matrix add seq 11000
matrix add seq 12000
matrix add seq 13000
matrix add seq 14000
matrix add seq 15000
matrix add seq 16000
matrix add seq 17000
matrix add seq 18000
matrix add seq 19000
matrix add seq 20000
matrix add seq 21000
matrix add seq 22000
matrix add seq 23000
matrix add seq 24000
matrix add seq 25000
matrix add seq 26000
matrix add seq 27000
matrix add seq 28000
matrix add seq 29000
matrix add seq 30000
matrix add seq 31000
matrix add seq 32000
matrix add seq 33000
matrix add seq 34000
matrix add seq 35000
matrix add seq 36000
matrix add seq 37000
matrix add seq 38000
matrix add seq 39000
matrix add seq 40000
matrix add seq 41000
matrix add seq 42000
matrix add seq 43000
matrix add seq 44000
matrix add seq 45000
matrix add seq 46000
matrix add seq 47000
matri

In [11]:
print(len(watch_seq))

69002


In [12]:
print(matrix['356'])

defaultdict(<class 'int'>, {'1195': 2582, '1535': 4730, '1575': 4879, '1604': 1196, '1840': 2274, '2167': 3567, '2904': 4479, '2993': 1693, '3455': 1227, '3712': 1817, '4181': 3165, '4214': 1337, '4224': 3668, '4654': 2309, '6201': 737, '6213': 1551, '6324': 712, '6347': 1976, '6547': 3965, '6702': 1876, '7054': 2023, '7592': 896, '7593': 971, '8074': 2753, '8516': 1368, '8525': 1952, '8536': 424, '8769': 1912, '8841': 1829, '8937': 1625, '9041': 1437, '9513': 1012, '9919': 2586, '10049': 793, '10080': 1725, '10213': 543, '10620': 2768, '10719': 1782, '10790': 1288, '10793': 2402, '11319': 1186, '11617': 1788, '11633': 1064, '11703': 533, '11743': 477, '11757': 3632, '11933': 988, '12049': 470, '13659': 1110, '13759': 1828, '14075': 1468, '14227': 1420, '14345': 1683, '14467': 1268, '14741': 1841, '14749': 714, '14813': 1763, '14967': 1394, '15225': 919, '15315': 1278, '15451': 1277, '15583': 1305, '15809': 2021, '16049': 959, '16157': 213, '16444': 288, '16498': 3352, '16512': 476, '1

In [13]:
## 概率
def get_transfer_prob(vs):
    neighbours = vs.keys()
    total_weight = sum(vs.values())
    
    probs = [vs[k]/total_weight for k in neighbours]
    try:
        assert( abs(1.0 - sum(probs)) < 0.000000001 )
    except:
        print(sum(probs))
        print(vs)
    
    return {
        'neighbours': list(neighbours),
        'probs': probs
    }



In [14]:
## 概率转移矩阵
# 3. build prob transfer matrix, for each key, each value <=> probability in total weights
transfer_probs = {k: get_transfer_prob(v) for k, v in matrix.items()}
print(f'transfer probs built. {len(transfer_probs)} entrances')

transfer probs built. 8265 entrances


In [15]:
transfer_probs['12049']

{'neighbours': ['356',
  '1195',
  '1535',
  '1575',
  '1604',
  '1840',
  '2167',
  '2904',
  '2993',
  '3455',
  '3712',
  '4181',
  '4214',
  '4224',
  '4654',
  '6201',
  '6213',
  '6324',
  '6347',
  '6547',
  '6702',
  '7054',
  '7592',
  '7593',
  '8074',
  '8516',
  '8525',
  '8536',
  '8769',
  '8841',
  '8937',
  '9041',
  '9513',
  '9919',
  '10049',
  '10080',
  '10213',
  '10620',
  '10719',
  '10790',
  '10793',
  '11319',
  '11617',
  '11633',
  '11703',
  '11743',
  '11757',
  '11933',
  '13659',
  '13759',
  '14075',
  '14227',
  '14345',
  '14467',
  '14741',
  '14749',
  '14813',
  '14967',
  '15225',
  '15315',
  '15451',
  '15583',
  '15809',
  '16049',
  '16157',
  '16444',
  '16498',
  '16512',
  '16668',
  '16706',
  '17651',
  '17741',
  '18857',
  '1',
  '15',
  '16',
  '20',
  '60',
  '68',
  '71',
  '73',
  '180',
  '181',
  '189',
  '195',
  '198',
  '205',
  '223',
  '225',
  '226',
  '237',
  '246',
  '355',
  '390',
  '469',
  '481',
  '502',
  '527',
  

In [19]:
# 4. entrance： 入口的构造
counts = {k: sum(neibors.values()) for k, neibors in matrix.items()}
entrance_items = list(transfer_probs.keys())    ## all keys
total_count = sum(counts.values())  ## all values => 加在一起
entrance_probs = [counts[k] / total_count for k in entrance_items]  ## each entrance probability
print(counts['6030'])   ## each entrance key: sum of values

# print(counts)
print(total_count)
print(entrance_probs[5])
# print(entrance_items)
print('entrance probs built.')

296925
620556990
0.0015106380479253001
entrance probs built.


In [20]:
## DEEP WALK N此采样，L长度的样本
## current_vertex ⇒ neighbors, probabilities ⇒ choose one of neighbors as current_vertex until choose 10 items
rng = np.random.default_rng()
## rng.choice(entrance_items, 1, p=entrance_probs): 
## This is a function call that randomly selects one element from entrance_items based on the probabilities provided in entrance_probs.
def one_random_walk(length, entrance_items, entrance_probs, transfer_probs):
    start_vertex = rng.choice(entrance_items, 1, p=entrance_probs)[0] ## 概率分布
    path = [str(start_vertex)]
    
    curr_vertex = start_vertex

    for _ in range(length):
        if curr_vertex not in transfer_probs:
            print(f"bad vertex {curr_vertex}")
            break
            
        neighbours = transfer_probs[curr_vertex]['neighbours']
        trans_prob = transfer_probs[curr_vertex]['probs']
        
        try:
            next_vertex = rng.choice(neighbours, 1, p=trans_prob)[0]
            path.append(str(next_vertex))

            curr_vertex = next_vertex
        except Exception as e:
            print(curr_vertex)
            print(e)
            break
        
    return path

In [21]:
n = 500
samples = [one_random_walk(20, entrance_items, entrance_probs, transfer_probs) for _ in range(n)] ## L=20

In [23]:
samples[:4]   ## 500 * 20

[['2926',
  '22547',
  '10020',
  '22319',
  '21327',
  '11499',
  '19953',
  '2451',
  '1519',
  '5593',
  '4901',
  '5681',
  '15199',
  '5081',
  '292',
  '6802',
  '5258',
  '1210',
  '1531',
  '11021',
  '2129'],
 ['8038',
  '12967',
  '4551',
  '3731',
  '18153',
  '478',
  '7791',
  '3228',
  '8675',
  '9736',
  '20583',
  '11887',
  '6351',
  '10793',
  '9201',
  '6811',
  '14075',
  '7054',
  '6512',
  '2142',
  '897'],
 ['4382',
  '7674',
  '202',
  '65',
  '73',
  '1535',
  '5114',
  '161',
  '1965',
  '392',
  '6573',
  '790',
  '13331',
  '13403',
  '10357',
  '9656',
  '5341',
  '18153',
  '901',
  '6',
  '5079'],
 ['10884',
  '3449',
  '18679',
  '6707',
  '16524',
  '5680',
  '1965',
  '6675',
  '27655',
  '355',
  '10408',
  '11737',
  '26243',
  '6045',
  '19815',
  '14345',
  '1689',
  '3299',
  '170',
  '30276',
  '10092']]

In [36]:

#### item2Vec

In [24]:
## array to dataframe
sample_df = spark.createDataFrame([[row] for row in samples], ['anime_ids'])

In [25]:
sample_df.show()

+--------------------+
|           anime_ids|
+--------------------+
|[2926, 22547, 100...|
|[8038, 12967, 455...|
|[4382, 7674, 202,...|
|[10884, 3449, 186...|
|[11761, 7724, 188...|
|[4182, 8408, 2104...|
|[1686, 356, 6324,...|
|[14027, 837, 2787...|
|[1538, 1562, 2789...|
|[11179, 16067, 31...|
|[20939, 23989, 21...|
|[1486, 31098, 200...|
|[10049, 4155, 312...|
|[21635, 6122, 192...|
|[732, 4901, 963, ...|
|[5277, 169, 237, ...|
|[28621, 28677, 14...|
|[11005, 1575, 989...|
|[1562, 12883, 593...|
|[411, 12413, 1535...|
+--------------------+
only showing top 20 rows



## The sequences of anime IDs represent paths taken in a graph of anime relationships, which could be based on user interactions, genre similarity, or some other metric. This gives context to each anime in relation to others.

In [26]:
from pyspark.ml.feature import Word2Vec    ## PySpark ML feature
##  Word2Vec turns words into fixed-size vectors (embeddings) where words with similar meanings are closer in the vector space.

item2vec = Word2Vec(vectorSize=5, maxIter=2, windowSize=15)   ## 参数比较小
item2vec.setInputCol('anime_ids')
item2vec.setOutputCol('anime_ids_vec')

model = item2vec.fit(sample_df)

rec = model.findSynonyms('20', 10) ## 找与‘20’最相似的十个
rec.show()

24/10/19 19:36:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/19 19:36:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


+-----+------------------+
| word|        similarity|
+-----+------------------+
|29095|0.9908428192138672|
| 8841|0.9657179713249207|
|30911|0.9582017064094543|
| 1699| 0.957165002822876|
| 1117|0.9567669034004211|
| 6895|0.9416693449020386|
|16009| 0.941557765007019|
|25681|0.9351367354393005|
|18247|0.9349497556686401|
|30187|0.9277344346046448|
+-----+------------------+



In [28]:
rec.show()

+-----+------------------+
| word|        similarity|
+-----+------------------+
|29095|0.9908428192138672|
| 8841|0.9657179713249207|
|30911|0.9582017064094543|
| 1699| 0.957165002822876|
| 1117|0.9567669034004211|
| 6895|0.9416693449020386|
|16009| 0.941557765007019|
|25681|0.9351367354393005|
|18247|0.9349497556686401|
|30187|0.9277344346046448|
+-----+------------------+



In [29]:
## User Embeddings: 根据用户看过的动漫的平均embedding => user的embedding

In [30]:
item_vec = model.getVectors().collect()
item_emb = {}
for item in item_vec:
    item_emb[item.word] = item.vector.toArray()

item_emb['20']

array([-0.20700456,  0.10432076,  0.05406615, -0.0613446 ,  0.10364742])

In [32]:
@udf(returnType='array<float>')
def build_user_emb(anime_seq):
    anime_embs = [item_emb[aid] if aid in item_emb else [] for aid in anime_seq]  ## ArrayType(ArrayType(FloatType()))
    anime_embs = list(filter(lambda l: len(l) > 0, anime_embs))
    ret = np.mean(anime_embs, axis=0).tolist()## ArrayType(FloatType()    ## axis=0 for each column
    return ret


anime_seq = ['123', '124', '423']
item_emb['123'] = [0.122, 0.144, 0.1441, 0.23, 0.23]


In [33]:
# user: [anime_ids]
user_emb_df = watch_seq_df.withColumn('user_emb', build_user_emb(col('anime_ids')))   ## extend dataframe new column: user_emb
first_row = user_emb_df.first()
print(first_row[1])
third_column_value = first_row[2]   ## userId, animeIds, userEmb
third_column_value


['8525', '10793', '19429']


[-0.19548577070236206,
 0.1619420349597931,
 0.0055765751749277115,
 0.1670580953359604,
 0.053637631237506866]

In [48]:
#### Save into Redis
# pip install redis

In [52]:
from redis import Redis

In [53]:
redis = Redis()

In [54]:
redis.hset('emb', '1', '[1,2,3,4]')

0

In [55]:
redis.hget('emb', '1')

b'[1,2,3,4]'

In [42]:
user_emb = user_emb_df.collect() ## to list
print(user_emb[0])
user_emb = {row.user_id: row.user_emb for row in user_emb} ### userId: userEmb
print(user_emb[26])


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt

Row(user_id=26, anime_ids=['8525', '10793', '19429'], user_emb=[-0.19548577070236206, 0.1619420349597931, 0.0055765751749277115, 0.1670580953359604, 0.053637631237506866])
[-0.19548577070236206, 0.1619420349597931, 0.0055765751749277115, 0.1670580953359604, 0.053637631237506866]


In [43]:
def vec2str(vec):
    if vec is None:    ## vector is None
        return '' 
    return ':'.join([str(v) for v in vec])      ## change to string "0.2":"0.321":"0.212":"0.312"

def save_user_emb(embs):
    str_emb = {item_id: vec2str(emb) for item_id, emb in embs.items()}
    redis.hset('recall-user-emb', mapping=str_emb)

save_user_emb(user_emb)


NameError: name 'redis' is not defined

In [58]:
redis.hget('recall-user-emb', '149')

b'0.14055117964744568:0.09811880439519882:0.10945238173007965:-0.10702409595251083:-0.0005224954802542925'

In [59]:
def str2vec(s):
    if len(s) == 0:
        return None
    return [float(x) for x in s.split(':')]

def load_user_emb():
    result = redis.hgetall('recall-user-emb')
    return {user_id.decode(): str2vec(emb.decode()) for user_id, emb in result.items()}   # a bytes-like object to string, save in Redis as bytes

In [60]:
load_user_emb()   ## get hashkey-value for key: 'recall-user-emb'

{'37223': [0.17545418441295624,
  0.08405015617609024,
  0.07844451814889908,
  -0.17876635491847992,
  0.032912321388721466],
 '11016': [0.11345811188220978,
  0.15407349169254303,
  0.09669800847768784,
  -0.15818729996681213,
  -0.013599478639662266],
 '20242': [0.15917494893074036,
  0.10507426410913467,
  0.11152499169111252,
  -0.156302347779274,
  0.008571507409214973],
 '2018': [0.1436552107334137,
  0.08937784284353256,
  0.0835360661149025,
  -0.12325581908226013,
  -0.0037575687747448683],
 '43303': [0.23294876515865326,
  0.0700758844614029,
  0.057012543082237244,
  -0.20577871799468994,
  -0.005018793512135744],
 '1518': [0.17870734632015228,
  0.10191015154123306,
  0.10090949386358261,
  -0.1884588897228241,
  -0.0037898554001003504],
 '61795': [0.16530796885490417,
  0.08195091038942337,
  0.10160109400749207,
  -0.15188734233379364,
  0.011513540521264076],
 '31318': [0.15793700516223907,
  0.07296410202980042,
  0.0784677118062973,
  -0.15835022926330566,
  0.0206673

In [61]:
### LSH 近似最近邻的检索

In [44]:
import faiss
import numpy as np

In [45]:
vecs = [
    [1, 0, 0],
    [0, 0, 0],
    [1, 1, 1]
]
vecs = np.asarray(vecs, dtype=np.float32)
vecs

array([[1., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.]], dtype=float32)

In [46]:
index = faiss.IndexLSH(3, 16)
index.add(vecs)

In [48]:
D, I = index.search(np.asarray([[1, 0, 0]], dtype=np.float32), 3)

In [49]:
D

array([[ 0.,  6., 10.]], dtype=float32)

In [50]:
I   ## 最近的三个row => 近似

array([[0, 2, 1]])

In [51]:
### LSH for item embeddings
## 有些item是没有embedding的，需要过滤 => filter()

In [52]:
item_emb ## dict

{'710': array([-0.09618831,  0.25195667, -0.03805553,  0.02045732,  0.11894488]),
 '21327': array([-0.08877359,  0.23027559,  0.00359384,  0.14011402, -0.10110765]),
 '205': array([-0.01201699,  0.178161  ,  0.04208241,  0.37097222,  0.00496736]),
 '45': array([ 0.04316349,  0.09901877, -0.05184022,  0.11785816,  0.03186139]),
 '20159': array([-0.11965747,  0.07629334, -0.01482891,  0.06147496, -0.07907193]),
 '1210': array([-0.14040785,  0.42264441,  0.05040532,  0.09915658, -0.04351611]),
 '28249': array([-0.26791033,  0.19129013,  0.1658069 , -0.03544264, -0.0283397 ]),
 '1690': array([-0.08510859,  0.13887903, -0.05487105,  0.11694714,  0.00764451]),
 '14349': array([-0.23011491,  0.43058622,  0.00433341,  0.10681234, -0.03494248]),
 '5030': array([-0.25317436,  0.18898818,  0.02388705,  0.15411703, -0.06114138]),
 '1965': array([-0.07957349,  0.22450396, -0.04849572,  0.01873834,  0.05593971]),
 '1699': array([-0.22207089,  0.17904384,  0.09573967, -0.0337607 ,  0.07452344]),
 '91

In [53]:
emb_items = item_emb.items()  ## (key, value)
emb_items

dict_items([('710', array([-0.09618831,  0.25195667, -0.03805553,  0.02045732,  0.11894488])), ('21327', array([-0.08877359,  0.23027559,  0.00359384,  0.14011402, -0.10110765])), ('205', array([-0.01201699,  0.178161  ,  0.04208241,  0.37097222,  0.00496736])), ('45', array([ 0.04316349,  0.09901877, -0.05184022,  0.11785816,  0.03186139])), ('20159', array([-0.11965747,  0.07629334, -0.01482891,  0.06147496, -0.07907193])), ('1210', array([-0.14040785,  0.42264441,  0.05040532,  0.09915658, -0.04351611])), ('28249', array([-0.26791033,  0.19129013,  0.1658069 , -0.03544264, -0.0283397 ])), ('1690', array([-0.08510859,  0.13887903, -0.05487105,  0.11694714,  0.00764451])), ('14349', array([-0.23011491,  0.43058622,  0.00433341,  0.10681234, -0.03494248])), ('5030', array([-0.25317436,  0.18898818,  0.02388705,  0.15411703, -0.06114138])), ('1965', array([-0.07957349,  0.22450396, -0.04849572,  0.01873834,  0.05593971])), ('1699', array([-0.22207089,  0.17904384,  0.09573967, -0.033760

In [54]:
emb_items = list(filter(lambda t: len(t[1]) > 0, emb_items))

In [55]:
emb_items

[('710',
  array([-0.09618831,  0.25195667, -0.03805553,  0.02045732,  0.11894488])),
 ('21327',
  array([-0.08877359,  0.23027559,  0.00359384,  0.14011402, -0.10110765])),
 ('205',
  array([-0.01201699,  0.178161  ,  0.04208241,  0.37097222,  0.00496736])),
 ('45',
  array([ 0.04316349,  0.09901877, -0.05184022,  0.11785816,  0.03186139])),
 ('20159',
  array([-0.11965747,  0.07629334, -0.01482891,  0.06147496, -0.07907193])),
 ('1210',
  array([-0.14040785,  0.42264441,  0.05040532,  0.09915658, -0.04351611])),
 ('28249',
  array([-0.26791033,  0.19129013,  0.1658069 , -0.03544264, -0.0283397 ])),
 ('1690',
  array([-0.08510859,  0.13887903, -0.05487105,  0.11694714,  0.00764451])),
 ('14349',
  array([-0.23011491,  0.43058622,  0.00433341,  0.10681234, -0.03494248])),
 ('5030',
  array([-0.25317436,  0.18898818,  0.02388705,  0.15411703, -0.06114138])),
 ('1965',
  array([-0.07957349,  0.22450396, -0.04849572,  0.01873834,  0.05593971])),
 ('1699',
  array([-0.22207089,  0.17904384

In [63]:
embs = [i[1] for i in emb_items]
embs[0]

array([-0.09618831,  0.25195667, -0.03805553,  0.02045732,  0.11894488])

In [57]:
index = faiss.IndexLSH(len(embs[0]), 256)
index.add(np.asarray(embs, dtype=np.float32))

In [58]:
D, I = index.search(np.asarray([embs[99]], dtype=np.float32), 10)

In [59]:
D

array([[ 0., 11., 14., 15., 16., 18., 19., 19., 20., 20.]], dtype=float32)

In [60]:
I  ## 相似的前十个

array([[ 99, 508, 455, 379, 386, 173,  15, 665, 479, 605]])

In [65]:
embs[99]
emb_items[99]   ## '343'

('343',
 array([-0.11579569,  0.10420679,  0.02577279,  0.15500808,  0.09854845]))

In [66]:
embs[108]
emb_items[108]   ## real id is '1889'

('1889',
 array([-0.18993649,  0.12439698,  0.15787438, -0.01632276, -0.14187722]))

In [6]:
gen_strs = ["Action, Adventure, Drama", "Comedy, Drama, School"]
gens = [s.split(",") for s in gen_strs]     ## [['Action', ' Adventure', ' Drama'], ['Comedy', ' Drama', ' School']]


print(gens)

gens = [x for l in gens for x in l] # flatten
gens = [s.strip() for s in gens]

gen_set = set(gens)
gen_set

[['Action', ' Adventure', ' Drama'], ['Comedy', ' Drama', ' School']]


{'Action', 'Adventure', 'Comedy', 'Drama', 'School'}